In [1]:
!echo $LD_LIBRARY_PATH

/usr/local/cuda-8.0/lib64:/usr/local/cuda-8.0/extras/CUPTI/lib64:/lib/nccl/cuda-8:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:


In [2]:
from tensorflow.python.client import device_lib

In [3]:
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13735331216015276412
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 8530905181530823260
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


In [4]:
import keras
from keras import callbacks
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

import sys
sys.path.insert(0, './road_simulator/src/')

from models.utils import get_datasets


paths = ['./datasets/dashed/']
n_images = 50 * 1000  # number of images
model_path = './models/my_autopilot_big.hdf5'

# Get train, test and val sets
train_X, train_Y, val_X, val_Y, test_X, test_Y = get_datasets(paths, n_images=n_images)

print('train_X[0]: ', train_X[0])
print('train_Y[0]: ', train_Y[0])


/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
  0%|          | 171/50000 [00:00<00:29, 1707.85it/s]

./datasets/dashed/


100%|██████████| 50000/50000 [00:26<00:00, 1882.68it/s]


train_X[0]:  [[[0.89411765 0.8666667  0.79607844]
  [0.7372549  0.70980394 0.6392157 ]
  [0.69411767 0.6666667  0.59607846]
  ...
  [0.9490196  0.9372549  0.87058824]
  [0.972549   0.9607843  0.89411765]
  [0.972549   0.9607843  0.89411765]]

 [[0.36078432 0.33333334 0.2627451 ]
  [0.54901963 0.52156866 0.4509804 ]
  [0.5372549  0.50980395 0.4392157 ]
  ...
  [0.95686275 0.94509804 0.8784314 ]
  [0.9647059  0.9529412  0.8862745 ]
  [0.9647059  0.9529412  0.8862745 ]]

 [[0.70980394 0.68235296 0.6117647 ]
  [0.7058824  0.6784314  0.60784316]
  [0.8117647  0.78431374 0.72156864]
  ...
  [0.9882353  0.9764706  0.9098039 ]
  [0.95686275 0.94509804 0.8784314 ]
  [0.95686275 0.94509804 0.8784314 ]]

 ...

 [[0.75686276 0.7137255  0.59607846]
  [0.6313726  0.5882353  0.47058824]
  [0.45490196 0.4117647  0.3019608 ]
  ...
  [0.5803922  0.5647059  0.5176471 ]
  [0.5411765  0.5254902  0.47843137]
  [0.54509807 0.5294118  0.48235294]]

 [[0.70980394 0.67058825 0.57254905]
  [0.6784314  0.6392157 

In [5]:
# Create the model

dropout_value = 0.1
input_shape = (70, 250, 3)

img_in = Input(shape=input_shape, name='img_in')
x = img_in

x = Convolution2D(5, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)
x = Dropout(dropout_value)(x)
x = BatchNormalization()(x)

x = Convolution2D(2, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)
x = Dropout(dropout_value)(x)
x = BatchNormalization()(x)

x = Convolution2D(2, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)
x = Dropout(dropout_value)(x)
x = BatchNormalization()(x)

x = Convolution2D(4, 3, 3, activation='relu', border_mode='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(x)
x = Dropout(dropout_value)(x)
x = BatchNormalization()(x)

flat = Flatten()(x)

x = Dense(20)(flat)
x = Activation('relu')(x)

x = Dense(5)(x)
angle_out = Activation('softmax')(x)

adam = Adam()
model = Model(input=[img_in], output=[angle_out])
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#Save the model after each epoch if the validation loss improved.
save_best = callbacks.ModelCheckpoint(model_path,
                                        monitor='val_loss',
                                        verbose=1,
                                        save_best_only=True,
                                        mode='min')
#stop training if the validation loss doesn't improve for 5 consecutive epochs.
early_stop = callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=2,
                                        verbose=0,
                                        mode='auto')
callbacks_list = [save_best, early_stop]


model.fit([train_X], train_Y,
            batch_size=100,
            nb_epoch=40,
            validation_data=([val_X], val_Y),
            callbacks=callbacks_list,
            verbose=2)

/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (3, 3), activation="relu", padding="same")`
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3), activation="relu", padding="same")`
  
/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3), activation="relu", padding="same")`
/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", padding="same")`
/home/ubuntu/anaconda3/envs/ironcar/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Mode

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_in (InputLayer)          (None, 70, 250, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 70, 250, 5)        140       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 125, 5)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 125, 5)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 35, 125, 5)        20        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 35, 125, 2)        92        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 62, 2)         0         
__________